# 网络IO

> 什么是IO？

IO 是 Input/Output 的缩写，指的是输入和输出。在计算机当中，IO 操作通常指将数据从一个设备或文件中读取到计算机内存中，或将内存中的数据写入设备或文件中。这些设备可以包括硬盘驱动器、网卡、键盘、屏幕等。

通常用户进程中的一个完整I/O分为两个阶段：

用户进程空间→内核空间

内核空间→设备空间

> I/O 的分类

1. 内存I/O
2. 网络I/O
3. 磁盘I/O

>  IO操作的两个阶段

Linux中进程无法直接操作I/O设备，其必须通过系统调用请求内核来协助完成I/O操作。

内核会为每个I/O设备维护一个缓冲区。对于一个输入操作来说，进程I/O系统调用后，内核会先看缓冲区中有没有相应的缓存数据，没有的话再到设备（比如网卡设备）中读取（因为设备I/O一般速度较慢，需要等待）；内核缓冲区有数据则直接复制到用户进程空间。
所以，对于一个网络输入操作通常包括两个不同阶段：

1. 等待网络数据到达网卡，把数据从网卡读取到内核缓冲区，准备好数据。
2. 从内核缓冲区复制数据到用户进程空间。

> 网络IO的本质

网络I/O的本质是对socket的读取，socket在Linux系统中被抽象为流，I/O可以理解为对流的操作。

> 网络I/O的模型可分为两种：

- 异步I/O(asynchronous I/O)
- 同步I/O(synchronous I/O)

同步I/O又包括

- 阻塞I/O(blocking I/O)
- 非阻塞I/O(non-blocking I/O)
- 多路复用I/O(multiplexing I/O)
- 信号驱动I/O(signal-driven I/O)

强调一下：信号驱动I/O属于同步I/O，原因往后看。



**信号驱动I/O和异步I/O只作概念性的讲解，不作为学习重点。**



## 五种I/O模型


对于一个套接字上的输入操作，第一步通常涉及等待数据从网络中到达，当所有等待分组到达时，它被复制到内核中的某个缓冲区。第二步是把数据从内核缓冲区复制到应用程序缓冲区。


> 阻塞I/O(blocking I/O)

同步阻塞I/O模型是最常用、最简单的模型。在Linux中，默认情况下，所有套接字都是阻塞的。下面我们以阻塞套接字的recvfrom的调用图来说明阻塞，如图所示

![alt text](阻塞IO.png)

**没准备好就休眠**

> 非阻塞I/O(non-blocking I/O)

非阻塞的recvform系统调用之后，进程并没有被阻塞，内核马上返回给进程，如果数据还没准备好，此时会返回一个`error`（`EAGAIN`或`EWOULDBLOCK`）

进程在返回之后，可以先处理其他的任务，稍后再发起recvform系统调用。
采用轮询的方式检查内核数据，直到数据准备好。再拷贝数据到进程，进行数据处理。
在Linux下，可以通过设置套接字选项使其变为非阻塞。

非阻塞的套接字的recvfrom操作如图所示

![alt text](非阻塞IO.png)

可以看到前三次调用recvfrom请求时，并没有数据返回，内核返回`errno`(`EWOULDBLOCK`)，并不会阻塞进程。
当第四次调用recvfrom时，数据已经准备好了，于是将它从内核空间拷贝到程序空间，处理数据。
在非阻塞状态下，I/O执行的等待阶段并不是完全阻塞的，但是第二个阶段依然处于一个阻塞状态（调用者将数据从内核拷贝到用户空间，这个阶段阻塞）。

**没准备好就返回错误**

> 多路复用I/O(multiplexing I/O)

I/O多路复用的好处在于单个进程就可以同时处理多个网络连接的I/O。它的基本原理是不再由应用程序自己监视连接，而由内核替应用程序监视文件描述符。

以select函数为例，当用户进程调用了select，那么整个进程会被阻塞，而同时，kernel会“监视”所有select负责的socket，当任何一个socket中的数据准备好，select就会返回。

这个时候用户进程再调用read操作，将数据从内核拷贝到用户进程，如下图所示。

![alt text](多路复用.png)

**select也阻塞，等待某个任务准备好**

> 信号驱动I/O(signal-driven I/O)

该模型允许socket进行信号驱动I/O，并注册一个信号处理函数，进程继续运行并不阻塞。当数据准备好时，进程会收到一个SIGIO信号，可以在信号处理函数中调用I/O操作函数处理数据，如图所示

![alt text](信号驱动IO.png)

注意：虽然信号驱动IO在注册完信号处理函数以后，就可以做其他事情了。但是第二阶段拷贝数据的过程当中进程依然是被阻塞的，而后要介绍的异步IO是完全不会阻塞进程的，所以信号驱动虽然具有异步的特点，但依然属于异步IO

> 异步I/O(asynchronous I/O)

相对于同步I/O，异步I/O不是按顺序执行。用户进程进行`aio_read`系统调用之后，就可以去处理其他逻辑了，无论内核数据是否准备好，都会直接返回给用户进程，不会对进程造成阻塞。这是因为`aio_read`只向内核递交申请，并不关心有没有数据。

等到数据准备好了，内核直接复制数据到进程空间，然后内核向进程发送通知，此时数据已经在用户空间了，可以对数据进行处理。

![alt text](异步IO.png)

> 五种I/O模型比较

![alt text](5种IO的比较.png)

前四种I/O模型都是同步I/O操作，它们的区别在于第一阶段，而第二阶段是一样的：在数据从内核复制到应用缓冲区期间（用户空间），进程阻塞于recvfrom调用。

相反，异步I/O模型在等待数据和接收数据的这两个阶段都是非阻塞的，可以处理其他的逻辑，用户进程将整个I/O操作交由内核完成，内核完成后会发送通知。在此期间，用户进程不需要检查I/O操作的状态，也不需要主动拷贝数据。

在了解了Linux的I/O模型之后，我们就可以进行服务器设计了。

# 多路复用IO的实现

多路复用的目的就是让单个进程就可以同时处理多个网络连接。基本原理是应用程序不自己监视连接，而由内核替应用程序监视文件描述符。
可以使用的函数有：  
+ select函数
+ poll函数
+ epoll 函数族

## select函数

使用select函数，当用户进程调用了select，那么整个进程会被阻塞，而同时，kernel会“监视”所有select负责的socket，当任何一个socket中的数据准备好，select就会返回。

select() 允许程序监控多个文件描述符，直到其中一个或多个文件描述符 "准备就绪"，可以进行某类 I/O 操作（如可能的输入）。  

如果可以执行相应的 I/O 操作（如输入），文件描述符就会被视为 "就绪"。I/O 操作不阻塞，则该文件描述符被视为准备就绪。  

select() 只能监控小于 FD_SETSIZE 的文件描述符编号；而 poll(2) 和 epoll(7) 则没有这个限制。

![alt text](select函数模型.png)

### 函数定义

```C
#include <sys/select.h>

int select(int nfds, fd_set* readfds, fd_set* writefds, fd_set* exceptfds, struct timeval* timeout);
```

> 函数参数

1. nfds 需要检查的文件描述符的数量  

   该参数应设置为所有`文件描述符`中编号最高的文件描述符，再加 1。检查每组中指定的文件描述符，最多不超过此限制

2. readfds 写文件描述符集合
3. writefds 读文件描述符集合  
   
   该文件描述符集合中的文件描述符会受到监视，以确定它们是否可以写入。如果写操作不会阻塞，文件描述符就可以写入。不过，即使文件描述符显示为可写，大量写入操作仍可能阻塞。  

   select() 返回后，writefds 将清除所有文件描述符，但已准备好写入的文件描述符除外。   

4. exceptfds  异常条件文件描述符集合  
   
   这组文件描述符会受到 "例外情况 "的监视

5. timeout  等待超时的时间  
   
   超时参数是一个 timeval 结构（如下所示），用于指定 select() 等待文件描述符就绪的阻塞时间间隔。
   如果设置为NULL，则select函数会一直等待

调用将阻塞，直到

- 文件描述符就绪；
- 调用被信号处理程序中断
- 超时。

请注意，超时时间间隔将四舍五入为系统时钟粒度。而内核调度延迟意味着阻塞间隔可能会超时一小段时间。

如果 timeval 结构的两个字段都为零，则 select()会立即返回(这对轮询很有用）

如果超时指定为 NULL，select() 将无限期阻塞等待文件描述符就绪。  

The timeout

```C
struct timeval {
      time_t      tv_sec;         /* seconds */
      suseconds_t tv_usec;        /* microseconds */
};


> 文件描述符集:  

select() 的主要参数是三个文件描述符 "集"（以 fd_set 类型声明），允许调用者在指定的文件描述符集上等待三类事件。  

![alt text](文件描述符集.png)

如果不需要监视文件描述符，则每个 fd_set 参数都可以指定为 NULL。如果没有文件描述符要被监视为相应类别的事件，则每个 fd_set 参数都可以指定为 NULL。

请注意：  

返回时，每个文件描述符集都会被就地修改，以表明哪些文件描述符当前 "就绪"。因此，如果在循环中使用 select()，每次调用前都必须`重新初始化`文件描述符集

将 fd_set 参数作为值-结果参数是一个设计错误，poll(2) 和 epoll(7) 避免了这一错误。

`fd_set` 是一个数据类型，用于表示文件描述符集合，在大多数系统中，`fd_set` 实际上是一个位数组（bit array），每一位对应一个文件描述符。但是，具体的实现可能因系统而异，因此我们通常不直接操作 `fd_set` 的内部结构，而是使用以下的宏来操作 `fd_set`：

文件描述符的内容可以通过以下宏来控制：

```C
       void FD_CLR(int fd, fd_set *set);
       int  FD_ISSET(int fd, fd_set *set);
       void FD_SET(int fd, fd_set *set);
       void FD_ZERO(fd_set *set);

```
`FD_ZERO()`  
    该宏用于`清除`（删除）集合中的所有文件描述符,它应作为初始化文件描述符集的第一步  
`FD_SET()`  
    此宏将文件描述符 fd `添加`到集合中。添加一个已经存在于集合中的文件描述符是无操作的，也不会产生错误  
`FD_CLR()`  
    此宏用于从集合中`删除`文件描述符 fd。删除集合中不存在的文件描述符是无操作的，不会产生错误  
`FD_ISSET()`  
    调用 select() 后，可以使用 FD_ISSET() 宏`测试`文件描述符是否仍然存在于集合中。 如果文件描述符 fd 存在于集合中，则 FD_ISSET() 返回非零；如果不存在，则返回零  

### 函数返回值
成功后，select() 和 pselect() 会返回三个返回的描述符集所包含的文件描述符集（即在 readfds、writefds、exceptfds 中被设置的位数）。
返回值可能为零，如果超时时间已过，但仍没有任何文件描述符准备就绪。脚本就绪前超时，返回值可能为零。
出错时，返回值为-1，并设置 Erno 表示出错；文件描述符集不会被修改,超时时间也不会被定义。

### select 应用实例

In [ ]:
// 头文件
#ifndef _NET_H_
#define _NET_H_

#include <stdio.h>
#include <stdlib.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/tcp.h>
#include <arpa/inet.h>
#include <unistd.h>
#include <strings.h>
#include <errno.h>

typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

#define BACKLOG 5  // 监听队列的最大长度
#define ErrExit(msg) do{perror(msg); exit(EXIT_FAILURE);} while(0) 
//该行代码的作用是：如果程序在执行过程中发生错误，会打印出错误信息，并退出程序

// 功能函数
void Argment(int argc, char *argv[]);  // 参数检查
int CreateSocket(char *argv[]);  // 创建套接字
int DataHandle(int fd); // 数据处理

#endif

In [ ]:
//函数定义
#include "net.h"

void Argment(int argc, char *argv[])
{
	if(argc < 3)
	{
		fprintf(stderr, "%s<addr><port>\n", argv[0]);
		exit(0);
	}
}

int CreateSocket(char *argv[])
{
	/*创建套接字*/
	int fd = socket(AF_INET, SOCK_STREAM, 0);
	if(fd < 0)
		ErrExit("socket");
	/*允许地址快速重用*/
	int flag = 1;
	if(setsockopt(fd, SOL_SOCKET, SO_REUSEADDR, &flag, sizeof(flag)))
		perror("setsockopt");
	/*设置通信结构体*/
	Addr_in addr;
	bzero(&addr, sizeof(addr));
	addr.sin_family = AF_INET;
	addr.sin_port = htons(atoi(argv[2]));
	/*绑定通信结构体*/
	if(bind(fd, (Addr *)&addr, sizeof(Addr_in)))
		ErrExit("bind");
	/*设置套接字为监听模式*/
	if(listen(fd, BACKLOG))
		ErrExit("listen");
	return fd;
}

int DataHandle(int fd)
{
	char buf[BUFSIZ] = {};
	Addr_in peeraddr;
	socklen_t peerlen = sizeof(Addr_in); 
	if(getpeername(fd, (Addr *)&peeraddr, &peerlen))//获取对端地址
		perror("getpeername");

	int ret = recv(fd, buf, BUFSIZ, 0);
	if(ret < 0)
		perror("recv");
	if(ret > 0)
	{
		printf("[%s:%d]data: %s\n", 
				inet_ntoa(peeraddr.sin_addr), ntohs(peeraddr.sin_port), 
				buf);//打印数据
	}
	return ret;
}

In [ ]:
#include "net.h"
#include <sys/select.h>
#define MAX_SOCK_FD 1024

int main(int argc, char *argv[])
{
	/*检查参数，小于3个 直接退出进程*/
	Argment(argc, argv);
	/*创建已设置监听模式的套接字*/
	int fd = CreateSocket(argv);
	
	/*设置文件描述符集合*/
	fd_set set, tmpset;  // 定义两个文件描述符集合， tmpset用于辅助
	FD_ZERO(&set);  // 清空文件描述符集合
	FD_ZERO(&tmpset);// 清空文件描述符集合
	FD_SET(fd, &set);// 将fd加入到文件描述符集合中
	
	int i, ret, newfd;
	Addr_in clientaddr;
	socklen_t clientlen = sizeof(Addr_in);
	
	while(1)
	{
		tmpset = set;
		if((ret = select(MAX_SOCK_FD, &tmpset, NULL, NULL, NULL)) < 0)
		//select函数用于监视文件描述符的变化情况
		//第一个参数是文件描述符的最大值加1
		//第二个参数是读文件描述符集合, 这里设置成tmpset是因为select会修改文件描述符集合
		//第三个参数是写文件描述符集合，这里设置成NULL是因为不关心写
		//第四个参数是异常文件描述符集合，这里设置成NULL是因为不关心异常
		//第五个参数是超时时间，这里设置成NULL是因为不关心超时
			ErrExit("select");
		if(FD_ISSET(fd, &tmpset))  //判断文件描述符是否在集合中,如果在则说明有新的连接
		{
			/*接收客户端连接，并生成新的文件描述符*/
			if((newfd = accept(fd, (Addr *)&clientaddr, &clientlen)) < 0)
				perror("accept");
			printf("[%s:%d]已建立连接\n", 
					inet_ntoa(clientaddr.sin_addr), ntohs(clientaddr.sin_port));
			FD_SET(newfd, &set);  //将新的文件描述符加入到文件描述符集合中
		}
		else
		{ //如果不是新的连接，则是已有连接的数据到达
			for(i = fd + 1; i < MAX_SOCK_FD; i++)  //遍历文件描述符集合
			{
				if(FD_ISSET(i, &tmpset))//判断文件描述符是否在集合中
				{
					if(DataHandle(i) <= 0)  //处理数据
					{
						if(getpeername(i, (Addr *)&clientaddr, &clientlen))//获取对端地址
							perror("getpeername");
						printf("[%s:%d]断开连接\n", 
								inet_ntoa(clientaddr.sin_addr), ntohs(clientaddr.sin_port));
						FD_CLR(i, &set);  //将断开的文件描述符从文件描述符集合中删除
					}
				}
			}
		}
	}
	return 0;
}

#### 思路详解


在你的代码中，已经创建了一个文件描述符集合`set`，并将监听套接字的文件描述符`fd`添加到了这个集合中。然后，你使用select函数来等待这个套接字上的连接请求：



In [ ]:
tmpset = set;
if((ret = select(MAX_SOCK_FD, &tmpset, NULL, NULL, NULL)) < 0) 
    ErrExit("select");



这段代码将`set`复制到`tmpset`，然后调用select函数等待`tmpset`中的任何一个文件描述符变得可读（即有连接请求）。如果select函数返回，那么`tmpset`中的文件描述符将被修改，只有那些变得可读的文件描述符仍然在集合中。

然后，你使用FD_ISSET宏检查监听套接字的文件描述符是否在`tmpset`中：



In [ ]:
if(FD_ISSET(fd, &tmpset))



如果在，那么说明有新的连接请求，你可以调用accept函数接受这个请求，并将返回的新的文件描述符添加到`set`中：



In [ ]:
if((newfd = accept(fd, (Addr *)&clientaddr, &clientlen)) < 0)
    perror("accept");
FD_SET(newfd, &set);



如果不在，那么说明是其他文件描述符变得可读，你可以遍历`tmpset`，处理这些文件描述符上的数据：



In [ ]:
for(i = fd + 1; i < MAX_SOCK_FD; i++)
{
    if(FD_ISSET(i, &tmpset))//判断文件描述符是否在集合中
    {
        if(DataHandle(i) <= 0)
        {
            FD_CLR(i, &set);
        }
    }
}



这段代码首先使用FD_ISSET宏检查文件描述符是否在`tmpset`中，如果在，那么调用DataHandle函数处理数据，如果DataHandle函数返回值小于等于0，那么使用FD_CLR宏将文件描述符从`set`中移除。

### getpeername

`getpeername` 是一个系统调用，用于获取与指定套接字关联的对端网络地址。这个函数在 C 程序中常用于获取已连接套接字的对端信息。

函数原型如下：



In [ ]:
int getpeername(int sockfd, struct sockaddr *addr, socklen_t *addrlen);



参数说明：

- `sockfd`：已连接的套接字描述符。
- `addr`：用于存储对端网络地址的结构体指针。
- `addrlen`：在调用前，这个参数应该被设置为`addr`所指向的缓冲区的大小。在调用后，这个参数会被设置为实际存储在`addr`中的地址的大小。

如果函数成功，返回0。如果出错，返回-1，并设置`errno`。

以下是一个使用 `getpeername` 的例子：



In [ ]:
#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>

int main() 
{
    int sockfd;
    struct sockaddr_in addr;
    socklen_t addrlen;

    // 假设 sockfd 是一个已连接的套接字

    addrlen = sizeof(addr);
    if (getpeername(sockfd, (struct sockaddr *)&addr, &addrlen) < 0) {
        perror("getpeername");
        return 1;
    }

    printf("Peer IP address: %s\n", inet_ntoa(addr.sin_addr));
    printf("Peer port: %d\n", ntohs(addr.sin_port));

    return 0;
}



这段代码首先调用 `getpeername` 获取与 `sockfd` 关联的对端网络地址，然后打印出这个地址的 IP 和端口。

## poll函数

`poll`函数是一种I/O多路复用的方法，它可以在多个文件描述符上等待一组事件（如读就绪、写就绪或异常）。

`poll`函数的原型如下：



```C
#include <poll.h>
int poll(struct pollfd* fds, nfds_t nfds, int timeout);
```

参数说明：

- `fds`：  
    
  一个指向`pollfd`结构体数组的指针，每个结构体用于指定一个文件描述符和你感兴趣的事件。

- `nfds`：  
   
  `fds`数组中的元素数量。

- `timeout`：  
  
    等待的超时时间，单位是毫秒。如果设置为-1，`poll`将无限等待。

`pollfd`结构体定义如下：

In [ ]:
struct pollfd {
    int   fd;         /* 文件描述符 */
    short events;     /* 请求的事件 */
    short revents;    /* 返回的事件 */
};



`events`和`revents`字段可以包含以下标志：

- `POLLIN`：数据可读。
- `POLLOUT`：数据可写。
- `POLLERR`：发生错误。

`poll`函数返回准备好的文件描述符数量，如果超时则返回0，如果出错则返回-1。

以下是一个使用`poll`函数的例子：



In [ ]:
#include <stdio.h>
#include <poll.h>

#define TIMEOUT 5

int main(void)
{
    struct pollfd fds[2];  //   文件描述符集合 结构体数组，这里设置以监听两种事件
    int ret;

    /* watch stdin for input */
    fds[0].fd = STDIN_FILENO; // 标准输入
    fds[0].events = POLLIN;  // 可读事件

    /* watch stdout for ability to write */
    fds[1].fd = STDOUT_FILENO; // 标准输出
    fds[1].events = POLLOUT; // 可写事件

    ret = poll(fds, 2, TIMEOUT * 1000);  // 5秒超时

    if (ret == -1) 
    {
        perror ("poll");
        return 1;
    }

    if (!ret) 
    {
        printf ("%d seconds elapsed.\n", TIMEOUT);
        return 0;
    }

    if (fds[0].revents & POLLIN)  // 判断是否可读 & 为位运算符 两个都为1 结果为1
        printf ("stdin is readable\n");

    if (fds[1].revents & POLLOUT)
        printf ("stdout is writable\n");

    return 0;
}



这段代码创建了一个`pollfd`数组，用于监视标准输入的读就绪事件和标准输出的写就绪事件，然后调用`poll`函数等待这些事件或超时。如果`poll`函数返回，代码将检查哪些事件已经准备好，并打印相应的消息。

In [ ]:
#ifndef _NET_H_
#define _NET_H_

#include <stdio.h>
#include <stdlib.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/tcp.h>
#include <arpa/inet.h>
#include <unistd.h>
#include <strings.h>
#include <errno.h>

typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;
#define BACKLOG 5
#define ErrExit(msg) do { perror(msg); exit(EXIT_FAILURE); } while(0)

void Argment(int argc, char *argv[]);
int CreateSocket(char *argv[]);
int DataHandle(int fd);


#endif

In [ ]:
#include "net.h"

void Argment(int argc, char *argv[])
{
	if(argc < 3)
	{
		fprintf(stderr, "%s<addr><port>\n", argv[0]);
		exit(0);
	}
}
int CreateSocket(char *argv[])
{
	/*创建套接字*/
	int fd = socket(AF_INET, SOCK_STREAM, 0);
	if(fd < 0)
		ErrExit("socket");
	/*允许地址快速重用*/
	int flag = 1;
	if( setsockopt(fd, SOL_SOCKET, SO_REUSEADDR, &flag, sizeof(flag)) )
		perror("setsockopt");
	/*设置通信结构体*/
	Addr_in addr;
	bzero(&addr, sizeof(addr) );
	addr.sin_family = AF_INET;
	addr.sin_port = htons( atoi(argv[2]) );
	/*绑定通信结构体*/
	if( bind(fd, (Addr *)&addr, sizeof(Addr_in)) )
		ErrExit("bind");
	/*设置套接字为监听模式*/
	if( listen(fd, BACKLOG) )
		ErrExit("listen");
	return fd;
}
int DataHandle(int fd)
{
	char buf[BUFSIZ] = {};
	Addr_in peeraddr;
	socklen_t peerlen = sizeof(Addr_in);
	if( getpeername(fd, (Addr *)&peeraddr, &peerlen) )
		perror("getpeername");
	int ret = recv(fd, buf, BUFSIZ, 0);
	if(ret < 0)
		perror("recv");
	if(ret > 0)
	{
		printf("[%s:%d]data: %s\n", 
				inet_ntoa(peeraddr.sin_addr), ntohs(peeraddr.sin_port), buf);
	}
	return ret;
}

In [ ]:
#include "net.h"
#include <poll.h>

#define MAX_SOCK_FD 1024
int main(int argc, char *argv[])
{
	int i, j, fd, newfd;
	nfds_t nfds = 1;  // 文件描述符个数
	struct pollfd fds[MAX_SOCK_FD] = {};  // pollfd结构体数组 用于存放文件描述符
	Addr_in addr; // 通信结构体
	socklen_t addrlen = sizeof(Addr_in);

	/*检查参数，小于3个 直接退出进程*/
	Argment(argc, argv);
	/*创建已设置监听模式的套接字*/
	fd = CreateSocket(argv);
	/*设置文件描述符集合*/
	fds[0].fd = fd;  // 将fd加入到文件描述符集合中
	fds[0].events = POLLIN;  // 设置监听事件 POLLIN 可读事件

	while(1)
	{
		if( poll(fds, nfds, -1) < 0) // -1表示永久阻塞  poll函数返回值为发生事件的文件描述符个数
			ErrExit("poll");
		for(i = 0; i < nfds; i++)//遍历文件描述符集合
		{
			/*接收客户端连接，并生成新的文件描述符*/
			if(fds[i].fd == fd && fds[i].revents & POLLIN) //第0个文件描述符 //  判断是否有新的连接
			{
				if( (newfd = accept(fd, (Addr *)&addr, &addrlen) ) < 0)
					perror("accept");
				fds[nfds].fd = newfd;  // 将newfd加入到文件描述符集合中
				fds[nfds].events = POLLIN;  // 设置监听事件
				nfds++;
				printf("[%s:%d][nfds=%lu] connection successful.\n", 
						inet_ntoa(addr.sin_addr), ntohs(addr.sin_port), nfds);
			}
			/*处理客户端数据*/
			if(i > 0 && fds[i].revents & POLLIN)
			{
				if(DataHandle(fds[i].fd) <= 0)
				{
					if( getpeername(fds[i].fd, (Addr *)&addr, &addrlen) < 0)
						perror("getpeername");
					printf("[%s:%d][fd=%d] exited.\n", 
							inet_ntoa(addr.sin_addr), ntohs(addr.sin_port), fds[i].fd);

					close(fds[i].fd);  // 关闭文件描述符
					for(j=i; j<nfds-1; j++) // 将fds[i]后面的元素向前移动
						fds[j] = fds[j+1];  // 将fds[i]后面的元素向前移动
					nfds--;
					i--;
				}
			}
		}
	}
	close(fd);
	return 0;
}

## epoll 函数

`epoll`是Linux特有的I/O多路复用机制，它包括三个主要的系统调用：`epoll_create1`，`epoll_ctl`和`epoll_wait`。

1. **epoll_create1**：这个函数用于创建一个新的epoll实例，并返回一个指向这个实例的文件描述符。

    ```c
    int epoll_create(int flags);
    ```

    `flags`参数可以设置为0或`EPOLL_CLOEXEC`。如果设置为`EPOLL_CLOEXEC`，则在执行新程序时将关闭epoll文件描述符。

2. **epoll_ctl**：这个函数用于向epoll实例中添加、修改或删除文件描述符。

    ```c
    int epoll_ctl(int epfd, int op, int fd, struct epoll_event *event);
    ```

    `epfd`是由`epoll_create1`返回的epoll文件描述符。`op`是要执行的操作，可以是`EPOLL_CTL_ADD`（添加）、`EPOLL_CTL_MOD`（修改）或`EPOLL_CTL_DEL`（删除）。`fd`是要操作的文件描述符。`event`是一个指向`epoll_event`结构体的指针，这个结构体指定了感兴趣的事件和与文件描述符关联的数据。

    `epoll_event`结构体定义如下：

    ```c
    struct epoll_event {
        uint32_t     events;  /* Epoll events */
        epoll_data_t data;    /* User data variable */
    };
    ```

    `events`字段可以包含以下标志：

    - `EPOLLIN`：数据可读。
    - `EPOLLOUT`：数据可写。
    - `EPOLLPRI`：有紧急数据可读（例如，TCP带外数据）。
    - `EPOLLERR`：发生错误。
    - `EPOLLHUP`：挂起。
    - `EPOLLET`：设置为边缘触发（Edge Triggered）模式，这是与水平触发（Level Triggered）模式相对的。

3. **epoll_wait**：这个函数等待epoll实例中的一个或多个事件。

    ```c
    int epoll_wait(int epfd, struct epoll_event *events, int maxevents, int timeout);
    ```

    `epfd`是由`epoll_create1`返回的epoll文件描述符。`events`是一个指向`epoll_event`结构体数组的指针，这个数组用于接收准备好的事件。`maxevents`是`events`数组的大小。`timeout`是等待的超时时间，单位是毫秒。如果设置为-1，`epoll_wait`将无限等待。

    `epoll_wait`返回准备好的事件数量，如果超时则返回0，如果出错则返回-1。

总的来说，`epoll`提供了一种高效的方式来处理大量并发的文件描述符。它只需要在文件描述符状态改变时才需要唤醒，而不是像`select`和`poll`那样需要不断轮询所有文件描述符。

epoll 函数实现实例

In [ ]:
#include "net.h"
#include <sys/epoll.h>

#define MAX_SOCK_FD 1024

int main(int argc, char *argv[])
{
	int i, nfds, fd, epfd, newfd;
	Addr_in addr;
	socklen_t addrlen = sizeof(Addr_in);
	struct epoll_event tmp, events[MAX_SOCK_FD] = {}; // epoll_event结构体数组 用于存放文件描述符
	/*检查参数，小于3个 直接退出进程*/
	Argment(argc, argv);
	/*创建已设置监听模式的套接字*/
	fd = CreateSocket(argv);

	if( (epfd = epoll_create(1)) < 0)  // 创建epoll句柄
		ErrExit("epoll_create");
	tmp.events = EPOLLIN;
	tmp.data.fd = fd;
	if( epoll_ctl(epfd, EPOLL_CTL_ADD, fd, &tmp) )  // 将fd加入到epoll句柄中
		ErrExit("epoll_ctl");

	while(1) 
    {
		if( (nfds = epoll_wait(epfd, events, MAX_SOCK_FD, -1) ) < 0)  // -1表示永久阻塞/*等待事件发生*/
			ErrExit("epoll_wait");
		printf("nfds = %d\n", nfds);

		for(i = 0; i < nfds; i++) 
        {
			if(events[i].data.fd == fd)   // 判断是否有新的连接
            {
				/*接收客户端连接，并生成新的文件描述符*/
				if( (newfd = accept(fd, (Addr *)&addr, &addrlen) ) < 0)
					perror("accept");
				printf("[%s:%d] connection.\n", inet_ntoa(addr.sin_addr), ntohs(addr.sin_port) );
				tmp.events = EPOLLIN;
				tmp.data.fd = newfd;
				if( epoll_ctl(epfd, EPOLL_CTL_ADD, newfd, &tmp) )
					ErrExit("epoll_ctl");
			}
            else
            {/*处理客户端数据*/
				if(DataHandle(events[i].data.fd) <= 0)
                {
					if( epoll_ctl(epfd, EPOLL_CTL_DEL, events[i].data.fd, NULL) )
						ErrExit("epoll_ctl");
					if( getpeername(events[i].data.fd, (Addr *)&addr, &addrlen) )
						perror("getpeername");
					
					printf("[%s:%d] exited.\n", inet_ntoa(addr.sin_addr), ntohs(addr.sin_port) );
					close(events[i].data.fd);
				}
			}
		}
	}
	close(epfd);
	close(fd);
	return 0;
}

# 套接字的属性

## 基本概念
- 设置套接字的选项对套接字进行控制
- 除了设置选项外，还可以获取选项
- 选项的概念相当于属性，所以套接字选项也可说是**套接字属性**
- 有些选项（属性）只可获取，不可设置；
- 有些选项既可设置也可获取
## 选项的级别
一些选项都是针对一种特定的协议
一些选项适用于所有类型的套接字
选项级别(level)的概念
### 常用的级别
| SOL_SOCKET | 该级别的选项只**作用于套接字本身** |
| --- | --- |
| SOL_LRLMP | 该级别的选项作用于**IrDA协议** |
| IPPROTO_IP | 该级别的选项作用于**IPv4协议** |
| IPPROTO_IPV6 | 该级别的选项作用于**IPv6协议** |
| IPPROTO_RM | 该级别的选项作用于可靠的多播传输 |
| IPPROTO_TCP | 该级别的选项适用于流式套接字 |
| IPPROTO_UDP | 该级别的选项适用于数据报套接字 |

### SOL_SOCKET的常用选项
| 选项名称 | 说明 | 获取/设置 |
| --- | --- | --- |
| SO_ACCEPTCONN | 套接字是否处于监听状态 | 获取 |
| SO_BROADCAST | 允许发送广播数据 | 两者都可 |
| SO_DEBUG | 允许调试 | 两者都可 |
| SO_DONTROUTE | 不查找路由 | 两者都可 |
| SO_ERROR | 获得套接字错误 | 获取 |
| SO_KEEPALIVE | 保活连接 | 两者都可 |
| SO_LINGER | 延迟关闭连接 | 两者都可 |
| SO_OOBINLINE | 带外数据放入正常数据流 | 两者都可 |
| SO_RCVBUF | 接收缓冲区大小 | 两者都可 |
| SO_SNDBUF | 发送缓冲区大小 | 两者都可 |
| SO_REUSERADDR | 允许重用本地地址和端口 | 两者都可 |
| SO_TYPE | 获得套接字类型 | 获取 |

### IPPROTO_IP级别的常用选项
| 选项名称 | 说明 | 获取/设置 |
| --- | --- | --- |
| IP_OPTIONS | 获取或设置IP头部内的选项 | 两者都可 |
| IP_HDRINCL | 是否将IP头部与数据一起提交给Winsock函数 | 两者都可 |
| IP_TTL | IP TTL相关 | 两者都可 |



## 获取套接字选项
### getsockopt函数
```c
#include <sys/types.h>          /* See NOTES */
#include <sys/socket.h>
int getsockopt(int sockfd, int level, int optname, void *optval, socklen_t *optlen);
```
#### 参数
sockfd：套接字描述符
level：表示选项的级别
optname：表示要获取的选项名称
optval：指向存放接收到的选项内容的缓冲区
optlen：指向optval所指缓冲区的大小
#### 函数返回值：
执行成功返回`0`，否则返回`‒1`，`errno`来获取错误码
#### 常见的错误码：
EBADF：参数sockfd不是有效的文件描述符
EFAULT：参数optlen太小或optval所指缓冲区非法
EINVAL：参数level未知或非法
ENOPROTOOPT：选项未知或不被指定的协议族所支持
ENOTSOCK：描述符不是一个套接字描述符
### 示例：获取流套接字和数据报套接字接收和发送的（内核）缓冲区大小

In [ ]:
#include <stdio.h>
#include <sys/time.h>
#include <sys/types.h>          
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <string.h>
#include <unistd.h>
#include <errno.h>
#include <sys/ioctl.h>
 
int main()
{
	int err,s = socket(AF_INET, SOCK_STREAM, 0);//创建流套接字
	if (s == -1) 
	{
		printf("Error at socket()\n");
		return -1;
	}
	int su = socket(AF_INET, SOCK_DGRAM, 0); //创建数据报套接字
	if (s == -1) 
	{
		printf("Error at socket()\n");
		return -1;
	}

	int optVal;
	int optLen = sizeof(optVal);
	//获取流套接字接收缓冲区大小
	if (getsockopt(s, SOL_SOCKET, SO_RCVBUF, (char*)&optVal,(socklen_t *)&optLen) == -1)
		printf("getsockopt failed:%d", errno);
	else
		printf("Size of stream socket receive buffer: %ld bytes\n", optVal);
	//获取流套接字发送缓冲区大小
	if (getsockopt(s, SOL_SOCKET, SO_SNDBUF, (char*)&optVal,(socklen_t *)&optLen) == -1)
		printf("getsockopt failed:%d", errno);
	else 
		printf("Size of streaming socket send buffer: %ld bytes\n", optVal);
	//获取数据报套接字接收缓冲区大小
	if (getsockopt(su, SOL_SOCKET, SO_RCVBUF, (char*)&optVal,(socklen_t *)&optLen) == -1)
		printf("getsockopt failed:%d", errno);
	else
		printf("Size of datagram socket receive buffer: %ld bytes\n", optVal);
	//获取数据报套接字发送缓冲区大小
	if (getsockopt(su, SOL_SOCKET, SO_SNDBUF, (char*)&optVal,(socklen_t *)&optLen) == -1)
		printf("getsockopt failed:%d", errno);
	else
		printf("Size of datagram socket send buffer:%ld bytes\n", optVal);

	getchar();
	return 0;
}

### 示例：获取当前套接字类型

In [ ]:
#include <stdio.h>
#include <assert.h>
#include <sys/time.h>
#include <sys/types.h>          
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <string.h>
#include <unistd.h>
#include <errno.h>
#include <sys/ioctl.h>

int main()
{
	int err;
	int s = socket(AF_INET, SOCK_STREAM, 0); //创建流套接字
	if (s == -1) {
		printf("Error at socket()\n");
		return -1;
	}
	int su = socket(AF_INET, SOCK_DGRAM, 0); //创建数据报套接字
	if (s == -1) {
		printf("Error at socket()\n");
		return -1;
	}

	int optVal;
	int optLen = sizeof(optVal);
	//获取套接字s的类型
	if (getsockopt(s, SOL_SOCKET, SO_TYPE, (char*)&optVal, (socklen_t *)&optLen) == -1)
		printf("getsockopt failed:%d", errno);
	else
	{
		if (SOCK_STREAM == optVal) // SOCK_STREAM宏定义值为1
			printf("The current socket is a stream socket.\n"); //当前套接字是流套接字
		else if (SOCK_DGRAM == optVal) // SOCK_ DGRAM宏定义值为2
			printf("The current socket is a datagram socket.\n");//当前套接字是数据报套接字
	}
	//获取套接字su的类型
	if (getsockopt(su, SOL_SOCKET, SO_TYPE, (char*)&optVal, (socklen_t *)&optLen) == -1)
		printf("getsockopt failed:%d", errno);
	else
	{
		if (SOCK_STREAM == optVal)  // SOCK_STREAM宏定义值为1
			printf("The current socket is a stream socket.\n");
		else if (SOCK_DGRAM == optVal) // SOCK_ DGRAM宏定义值为2
			printf("The current socket is a datagram socket.\n");
	}
	getchar();
	return 0;
}

### 示例：判断套接字是否处于监听状态

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <sys/time.h>
#include <sys/types.h>          
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <string.h>
#include <unistd.h>
#include <errno.h>
#include <sys/ioctl.h>
 
typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

#define ErrExit(msg) do { perror(msg); exit(EXIT_FAILURE); } while (0)

int main(int argc, char *argv[])
{
        Addr_in service;
        if(argc < 3)
        {
                printf("%s[ADDR][PORT]\n", argv[0]);
                exit(0);
        }
        int s = socket(AF_INET, SOCK_STREAM, 0); //创建一个流套接字
        if (s == -1) 
                ErrExit("socket");
        //允许地址的立即重用
        char on = 1;
        setsockopt(s, SOL_SOCKET, SO_REUSEADDR, &on, sizeof(on));

        service.sin_family = AF_INET;
        service.sin_addr.s_addr = inet_addr(argv[1]);
        service.sin_port = htons( atoi(argv[2]) );
        if (bind(s, (Addr*)&service, sizeof(service)) == -1) //绑定套接字
                ErrExit("bind");
        int optVal;
        int optLen = sizeof(optVal);
        //获取选项SO_ACCEPTCONN的值
        if (getsockopt(s, SOL_SOCKET, SO_ACCEPTCONN, (char*)&optVal, (socklen_t*)&optLen) == -1)
                printf("getsockopt failed:%d",errno);
        else printf("Before listening, The value of SO_ACCEPTCONN:%d, The socket is not listening\n", optVal);

        // 开始侦听
        if (listen(s, 100) == -1)
                ErrExit("listen");
        //获取选项SO_ACCEPTCONN的值
        if (getsockopt(s, SOL_SOCKET, SO_ACCEPTCONN, (char*)&optVal, (socklen_t*)&optLen) == -1)
                ErrExit("getsockopt");
        else printf("After listening,The value of SO_ACCEPTCONN:%d, The socket is listening\n", optVal);
        return 0;
}

## 设置套接字选项
### setsockopt函数
```c
int setsockopt(int sockfd, int level, int optname, const void *optval, socklen_t optlen);
```
#### 参数
sockfd：套接字描述符
level：表示选项的级别
optname：表示要获取的选项名称
optval：指向存放接收到的选项内容的缓冲区
optlen：指向optval所指缓冲区的大小
#### 函数返回值：
执行成功返回`0`，否则返回`‒1`，`errno`来获取错误码
### 示例：启用套接字的保活机制

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <sys/time.h>
#include <sys/types.h>          
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include <string.h>
#include <unistd.h>
#include <errno.h>
#include <sys/ioctl.h>
 
typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

#define ErrExit(msg) do { perror(msg); exit(EXIT_FAILURE); } while (0)

int main(int argc, char *argv[])
{
        Addr_in service;
        if(argc < 3)
        {
                printf("%s[ADDR][PORT]\n", argv[0]);
                exit(0);
        }

        int s = socket(AF_INET, SOCK_STREAM, 0); //创建一个流套接字
        if( s < 0)
                ErrExit("socket");

        char on = 1;
        setsockopt(s, SOL_SOCKET, SO_REUSEADDR, &on, sizeof(on));

        service.sin_family = AF_INET;
        service.sin_addr.s_addr = inet_addr(argv[1]);
        service.sin_port = htons(atoi(argv[2]));
        if (bind(s, (Addr *) &service, sizeof(service)) == -1) //绑定套接字
                ErrExit("bind");

        int optVal = 1;//一定要初始化
        int optLen = sizeof(int);

        //获取选项SO_KEEPALIVE的值
        if (getsockopt(s, SOL_SOCKET, SO_KEEPALIVE, (char*)&optVal, (socklen_t *)&optLen) == -1)
                ErrExit("getsockopt");
        else printf("After listening,the value of SO_ACCEPTCONN:%d\n", optVal);

        optVal = 1;
        if (setsockopt(s, SOL_SOCKET, SO_KEEPALIVE, (char*)&optVal, optLen) != -1)
                printf("Successful activation of keep alive mechanism.\n");//启用保活机制成功

        if (getsockopt(s, SOL_SOCKET, SO_KEEPALIVE, (char*)&optVal, (socklen_t *)&optLen) == -1)
                ErrExit("getsockopt");
        else printf("After setting,the value of SO_KEEPALIVE:%d\n", optVal);

        return 0;
}

### 示例： 设置保活连接

In [ ]:
#include "net.h"
#include <sys/select.h>
#define MAX_SOCK_FD 1024

void setKeepAlive (int sockfd, int attr_on, socklen_t idle_time, socklen_t interval, socklen_t cnt)
{
	setsockopt (sockfd, SOL_SOCKET, SO_KEEPALIVE, (const char *) &attr_on, sizeof (attr_on));
	setsockopt (sockfd, SOL_TCP, TCP_KEEPIDLE, (const char *) &idle_time, sizeof (idle_time));
	setsockopt (sockfd, SOL_TCP, TCP_KEEPINTVL, (const char *) &interval, sizeof (interval));
	setsockopt (sockfd, SOL_TCP, TCP_KEEPCNT, (const char *) &cnt, sizeof (cnt));
}

int main(int argc, char *argv[])
{
	int i, ret, fd, newfd;
	fd_set set, tmpset;
	Addr_in clientaddr;
	socklen_t clientlen = sizeof(Addr_in);
	/*检查参数，小于3个 直接退出进程*/
	Argment(argc, argv);
	/*创建已设置监听模式的套接字*/
	fd = CreateSocket(argv);

	FD_ZERO(&set);
	FD_ZERO(&tmpset);
	FD_SET(fd, &set);

	while(1)
    {
		tmpset = set;
		if((ret = select(MAX_SOCK_FD, &tmpset, NULL, NULL, NULL)) < 0)
        {
			perror("select");
			getchar();
		}
		if(FD_ISSET(fd, &tmpset))
        {
			/*接收客户端连接，并生成新的文件描述符*/
			if( (newfd = accept(fd, (Addr *)&clientaddr, &clientlen) ) < 0)
            {
				perror("accept");
				getchar();
			}
#if 1
			int keepAlive = 1;			//设定KeepAlive
			int keepIdle = 5;			//开始首次KeepAlive探测前的TCP空闭时间
			int keepInterval = 5;		//两次KeepAlive探测间的时间间隔
			int keepCount = 3;			//判定断开前的KeepAlive探测次数
			setKeepAlive (newfd, keepAlive, keepIdle, keepInterval, keepCount);
#endif

			printf("[%s:%d]已建立连接\n", 
					inet_ntoa(clientaddr.sin_addr), ntohs(clientaddr.sin_port));
			FD_SET(newfd, &set);
		}
        else
        { //处理客户端数据
			for(i = fd + 1; i < MAX_SOCK_FD; i++)
            {
				if(FD_ISSET(i, &tmpset))
                {
					if( DataHandle(i) <= 0)
                    {
						if(getpeername(i, (Addr *)&clientaddr, &clientlen))
							perror("getpeername");
						printf("[%s:%d]断开连接\n", 
								inet_ntoa(clientaddr.sin_addr), ntohs(clientaddr.sin_port));
						FD_CLR(i, &set);
						close(i);
					}
				}
			}
		}
	}
	close(fd);
	return 0;
}

# 广播

## 广播实现

发送端

In [ ]:
#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/ip.h> /* superset of previous */
#include <unistd.h>
#include <stdlib.h>
#include <arpa/inet.h>
#include <strings.h>
#include <string.h>

#define ErrExit(msg) do {perror(msg); exit(EXIT_FAILURE);} while(0)
typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

int main(int argc, char *argv[])
{
	int fd = -1;
	Addr_in peeraddr;
	socklen_t peerlen = sizeof(peeraddr);
	char buf[BUFSIZ] = {};
	/*参数检查*/
	if(argc < 3)
    {
		fprintf(stderr, "%s<multiaddr><port>", argv[0]);
		exit(EXIT_FAILURE);
	}
	/*创建套接字*/
	if( (fd = socket(AF_INET, SOCK_DGRAM, 0) ) < 0)
		ErrExit("socket");

	/*允许广播*/
	int on = 1;
	setsockopt(fd, SOL_SOCKET, SO_BROADCAST, &on, sizeof(on));

	/*设置通信结构体*/
	peeraddr.sin_family = AF_INET;
	peeraddr.sin_port = htons(atoi(argv[2]));
	if(!inet_aton(argv[1], &peeraddr.sin_addr) )
    {
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}
	while(1)
    {
		fgets(buf, BUFSIZ, stdin);
		sendto(fd, buf, strlen(buf)+1, 0, (Addr *)&peeraddr, peerlen);
	}
	return 0;
}

接收端

In [ ]:
#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/ip.h> /* superset of previous */
#include <unistd.h>
#include <stdlib.h>
#include <arpa/inet.h>
#include <strings.h>

#define ErrExit(msg) do {perror(msg); exit(EXIT_FAILURE);} while(0)
typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

int main(int argc, char *argv[])
{
	int fd = -1;
	Addr_in myaddr, peeraddr;
	socklen_t peerlen = sizeof(peeraddr);
	char buf[BUFSIZ] = {};
	/*参数检查*/
	if(argc < 3)
    {
		fprintf(stderr, "%s<addr><port>", argv[0]);
		exit(EXIT_FAILURE);
	}
	/*创建套接字*/
	if( (fd = socket(AF_INET, SOCK_DGRAM, 0) ) < 0)
			ErrExit("socket");
	/*设置通信结构体*/
	myaddr.sin_family = AF_INET;
	myaddr.sin_port = htons(atoi(argv[2]));
	if(!inet_aton(argv[1], &myaddr.sin_addr) )
    {
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}
	/*绑定通信结构体*/
	if( bind(fd, (Addr *)&myaddr, sizeof(Addr_in)) )
		ErrExit("bind");
	while(1)
    {
		recvfrom(fd, buf, BUFSIZ, 0, (Addr *)&peeraddr, &peerlen);
		printf("[%s:%d]%s\n", inet_ntoa(peeraddr.sin_addr), ntohs(peeraddr.sin_port), buf);
	}
	return 0;
}

# 组播

发送端

In [ ]:
#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/ip.h> /* superset of previous */
#include <unistd.h>
#include <stdlib.h>
#include <arpa/inet.h>
#include <strings.h>
#include <string.h>

#define ErrExit(msg) do {perror(msg); exit(EXIT_FAILURE);} while(0)
typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

int main(int argc, char *argv[])
{
	int fd = -1;
	Addr_in peeraddr;
	socklen_t peerlen = sizeof(peeraddr);
	char buf[BUFSIZ] = {};
	/*参数检查*/
	if(argc < 3){
		fprintf(stderr, "%s<multiaddr><port>", argv[0]);
		exit(EXIT_FAILURE);
	}
	/*创建套接字*/
	if( (fd = socket(AF_INET, SOCK_DGRAM, 0) ) < 0)
		ErrExit("socket");

	/*设置通信结构体*/
	peeraddr.sin_family = AF_INET;
	peeraddr.sin_port = htons(atoi(argv[2]));
	if(!inet_aton(argv[1], &peeraddr.sin_addr) ){
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}
	while(1){
		fgets(buf, BUFSIZ, stdin);
		sendto(fd, buf, strlen(buf)+1, 0, (Addr *)&peeraddr, peerlen);
	}
	return 0;
}

接收端

In [ ]:
#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/ip.h> /* superset of previous */
#include <unistd.h>
#include <stdlib.h>
#include <arpa/inet.h>
#include <strings.h>

#define ErrExit(msg) do {perror(msg); exit(EXIT_FAILURE);} while(0)
typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

int main(int argc, char *argv[])
{
	int fd = -1;
	Addr_in myaddr, peeraddr;
	socklen_t peerlen = sizeof(peeraddr);
	struct ip_mreqn mreq;
	char buf[BUFSIZ] = {};
	/*参数检查*/
	if(argc < 3){
		fprintf(stderr, "%s<addr><port>", argv[0]);
		exit(EXIT_FAILURE);
	}
	/*创建套接字*/
	if( (fd = socket(AF_INET, SOCK_DGRAM, 0) ) < 0)
			ErrExit("socket");
	/*加入多播组*/
	bzero(&mreq, sizeof(mreq) );
	if(!inet_aton(argv[1], &mreq.imr_multiaddr) ){
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}
	if(setsockopt(fd, IPPROTO_IP, IP_ADD_MEMBERSHIP, &mreq, sizeof(mreq)) < 0){
		perror("setsockopt");
		exit(0);
	}

	/*设置通信结构体*/
	myaddr.sin_family = AF_INET;
	myaddr.sin_port = htons(atoi(argv[2]));
	if(!inet_aton(argv[1], &myaddr.sin_addr) ){
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}
	/*绑定通信结构体*/
	if( bind(fd, (Addr *)&myaddr, sizeof(Addr_in)) )
		ErrExit("bind");
	while(1){
		recvfrom(fd, buf, BUFSIZ, 0, (Addr *)&peeraddr, &peerlen);
		printf("[%s:%d]%s\n", inet_ntoa(peeraddr.sin_addr), ntohs(peeraddr.sin_port), buf);
	}
	return 0;
}

# 原始套接字

In [ ]:
#include <stdio.h>
#include <sys/socket.h>
#include <netinet/ether.h>
#include <arpa/inet.h>
#include <unistd.h>
#include <arpa/inet.h>
#include <linux/ip.h>
#include <linux/tcp.h>
#include <net/ethernet.h>

#define MTU 1500

int main()
{
	/* 定义变量 */
	int sockfd, len;
	uint8_t buf[MTU]={};
	uint16_t ether_type;

	struct iphdr *iph;  //IP包头
	struct tcphdr *tcph;//TCP包头
	struct ether_header *eth;

	/* 创建一个链路层原始套接字 */
	if((sockfd = socket(PF_PACKET, SOCK_RAW, htons(ETH_P_ALL))) < 0)
	{
		perror("socket");
		return 0;
	}
	printf("sockfd = %d\n", sockfd);

	/* 接收(只接收TCP数据协议)并处理IP数据报 */
	while(1)
	{
		/* 接收包含TCP协议的IP数据报 */
		len = recvfrom(sockfd, buf, sizeof(buf),0,NULL,NULL);

		eth = (struct ether_header *)buf;
		ether_type = htons(eth->ether_type);
		switch(ether_type)
		{
		case ETHERTYPE_IP:
			printf("IP协议\n");
			break;
		case ETHERTYPE_ARP:
			printf("ARP协议\n");
			break;
		case ETHERTYPE_LOOPBACK:
			printf("loop back\n");
			break;
		default:
			printf("其他协议 %x\n",eth->ether_type);
		}
		if(ether_type != ETHERTYPE_IP)
			continue;

		/* 打印源IP和目的IP */
		iph = (struct iphdr *)(buf+14);
		if(iph->protocol != IPPROTO_TCP)
			continue;
		printf("源IP:%s\n",inet_ntoa(*(struct in_addr *)&iph->saddr) );
		printf("目的IP%s\n",inet_ntoa(*(struct in_addr *)&iph->daddr) );

		/* 打印TCP包头的源端口号和目的端口号 */
		tcph = (struct tcphdr *)(buf+14+iph->ihl*4);
		printf("%hu--->", ntohs(tcph->source));
		printf("%hu\n", ntohs(tcph->dest));

		/* 打印TCP数据段的长度 */
		printf("TCP首部长度:%d\n", tcph->doff*4);
	}
	//关闭套接字
	close(sockfd);
	return 0;
}

网络层原始套接字

In [ ]:
#include <stdio.h>
#include <sys/socket.h>
#include <netinet/ether.h>
#include <arpa/inet.h>
#include <unistd.h>
#include <arpa/inet.h>
#include <linux/ip.h>
#include <linux/tcp.h>
#include <sys/ioctl.h>
 
#define MTU 1500
 
int main()
{
    /* 定义变量 */
    int sockfd = -1, len, datalen, i;
    uint8_t buf[MTU]={}, *data;
 
    struct iphdr *iph;  //IP包头
    struct tcphdr *tcph;//TCP包头
    struct winsize size;
 
    /* 创建一个原始套接字 */
    if( (sockfd = socket(AF_INET, SOCK_RAW, IPPROTO_TCP) ) < 0)
    {
        perror("socket");
        return 0;
    }
    printf("sockfd = %d\n", sockfd);
 
    /* 接收(只接收TCP数据协议)并处理IP数据报 */
    while(1)
    {
        /* 接收包含TCP协议的IP数据报 */
        len = recvfrom(sockfd, buf, sizeof(buf),0,NULL,NULL);
        printf("IP数据报长度 = %d\n", len);
 
        /* 打印源IP和目的IP */
        iph = (struct iphdr *)buf;
        printf("源IP:%s",inet_ntoa(*(struct in_addr *)&iph->saddr) );
        printf("目的IP%s\n",inet_ntoa(*(struct in_addr *)&iph->daddr) );
 
        /* 打印TCP包头的源端口号和目的端口号 */
        tcph = (struct tcphdr *)(buf+iph->ihl*4);
        printf("%hu--->", ntohs(tcph->source));
        printf("%hu\n", ntohs(tcph->dest));
 
        /* 打印TCP数据段的长度 */
        printf("TCP首部长度:%d\n", tcph->doff*4);
        if(iph->ihl*4+tcph->doff*4 < len) 
        {
            data = buf + iph->ihl*4 + tcph->doff*4;
            datalen = len - iph->ihl*4 - tcph->doff*4;
            ioctl(STDIN_FILENO,TIOCGWINSZ,&size); //terminal 结构体
            for(i = 0; i < size.ws_col; i++) //显示一行 = 
                putchar('=');
            putchar('\n');
            printf("TCP数据字符:\n");
            for(i = 0; i < size.ws_col; i++)
                putchar('=');
            putchar('\n');
            for(i = 0; i < datalen-1; i++) 
            {
                printf("%c", data[i]);
            }
            for(i = 0; i < size.ws_col; i++)
                putchar('=');
            putchar('\n');
            printf("TCP数据16进制:\n");
            for(i = 0; i < size.ws_col; i++)
                putchar('=');
            putchar('\n');
            for(i = 0; i < datalen-1; i++)
            {
                printf("%x ", data[i]);
            }
            putchar('\n');
            for(i = 0; i < size.ws_col; i++)
                putchar('=');
            putchar('\n');
        }
    }
    //关闭套接字
    close(sockfd);
    return 0;
}

# http 服务器的实现

In [ ]:
#include <stdio.h>
#include <sys/socket.h>
#include <sys/types.h>
#include <stdlib.h>
#include <arpa/inet.h>
#include <unistd.h>
#include <errno.h>
#include <fcntl.h>
#include <sys/stat.h>
#include <strings.h>

#define PORT 80
#define BACKLOG 5
#define HTTPFILE "http-head.txt"
#define HTMLFILE "home.html"

int ClientHandle(int newfd);

int main(int argc, char *argv[])
{
	int fd, newfd;
	struct sockaddr_in addr;
	/*创建套接字*/
	fd = socket(AF_INET, SOCK_STREAM, 0);
	if(fd < 0)
	{
		perror("socket");
		exit(0);
	}
	int opt = 1;
	if(setsockopt(fd, SOL_SOCKET, SO_REUSEADDR, (const void *) &opt, sizeof(opt) ))
		perror("setsockopt");

	addr.sin_family = AF_INET;
	addr.sin_port = htons(PORT);
	addr.sin_addr.s_addr = 0;
	/*绑定通信结构体*/
	if(bind(fd, (struct sockaddr *)&addr, sizeof(addr) ) == -1){
		perror("bind");
		exit(0);
	}
	/*设置套接字为监听模式*/
	if(listen(fd, BACKLOG) == -1){
		perror("listen");
		exit(0);
	}
	/*接受客户端的连接请求，生成新的用于和客户端通信的套接字*/
	newfd = accept(fd, NULL, NULL);
	if(newfd < 0){
		perror("accept");
		exit(0);
	}
	ClientHandle(newfd);
	close(fd);
	return 0;
}

int ClientHandle(int newfd){
	int file_fd = -1;
	char buf[BUFSIZ] = {};
	int ret;

	do {
		ret = recv(newfd, buf, BUFSIZ, 0);
	}while(ret < 0 && errno == EINTR);
	if(ret < 0){
		perror("recv");
		exit(0);
	}else if(ret == 0){
		close(newfd);
		return 0;
	}else{
		printf("=====================================\n");
		printf("%s", buf);
		fflush(stdout);
	}

	bzero(buf, ret);
	file_fd = open(HTTPFILE, O_RDONLY);
	if(file_fd < 0){
		perror("open");
		exit(0);
	}
	ret = read(file_fd, buf, BUFSIZ);
	printf("%s\n", buf);
	send(newfd, buf, ret, 0);
	close(file_fd);

	bzero(buf, ret);
	file_fd = open(HTMLFILE, O_RDONLY);
	if(file_fd < 0){
		perror("open");
		exit(0);
	}
	ret = read(file_fd, buf, BUFSIZ);
	printf("%s\n", buf);
	send(newfd, buf, ret, 0);
	close(file_fd);

	close(newfd);
	return 0;
}

In [ ]:
HTTP/1.1 200 OK
Content-Type: text/html
Connection: close

In [ ]:
<html>
<head>
<title>homework</title>
<meta charset="utf-8">
</head>
<body>
<body>
<h1>服务器</h1>
<p>Hello World!</p>
</body>
</body>
</html>